In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [2]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

.\data\G04_FoG_trial_1_emg.csv
.\data\G04_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_1_emg.csv
.\data\G06_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_3_emg.csv
.\data\G07_Freezing_Trial1_trial_1_emg.csv
.\data\G08_FoG_1_trial_1_emg.csv
.\data\G08_FoG_2_trial_1_emg.csv
.\data\G11_FoG_trial_1_emg.csv
.\data\G11_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_3_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_3_emg.csv
.\data\P551_M050_2_A_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_2_emg.csv
.\data\P812_M050_2_B_FoG_trial_1_emg.csv
.\data\P812_M050_2_B_FoG_trial_2_emg.csv
.\data\其他\labels.txt
.\data\其他\P812_M050_2_B_FoG_trial1_annotation.csv
.\data\其他\P812_M050_2_B_FoG_trials.mat
.\data\其他\P812_M050_2_B_FoG_trial_1_out_left_foot.csv
.\data\其他\P812_M050_2_B_FoG_trial_1

In [72]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [51]:
Data = pd.read_csv('./processed data/dataframe_W256_S64_dropna.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'P551_M050_2_B_FoG_trial_2_emg.csv'
ind_drop = df.columns!=drop
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin(files)
Data_sel = Data[ind]
Data_rest = Data[~ind]

In [52]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('MAV')
#ind_temp2 = feature_all.columns.str.contains('RMS')
#ind_temp3 = feature_all.columns.str.contains('VAR')
#ind_temp = ind_temp1|ind_temp2|ind_temp3
#ind_temp = feature_all.columns.str.contains('mDWT')
feature = feature_all.loc[:,~ind_temp]
#feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
feature2_all = Data_rest.iloc[:,1:-1]
feature2 = feature2_all.loc[:,~ind_temp]
#feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [4]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [211]:
import imp
imp.reload(dp)

<module 'data_processing' from 'E:\\Document\\jupyter\\Master Thesis\\data_processing.py'>

In [66]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [23]:
feature

,Label,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,...,RIGHT_BF_mDWT5,RIGHT_BF_mDWT6,RIGHT_BF_mDWT7,RIGHT_RF_mDWT0,RIGHT_RF_mDWT1,RIGHT_RF_mDWT2,RIGHT_RF_mDWT3,RIGHT_RF_mDWT4,RIGHT_RF_mDWT5,RIGHT_RF_mDWT6
0,0,540.0,454.0,748.0,620.0,1406.0,4458.0,1992.0,887.0,2070.0,...,2262.686279,1259.309937,661.646484,627.024109,422.331390,325.584045,149.540817,34.614529,15.235371,7.567358
1,0,540.0,476.0,794.0,647.0,1235.0,3225.0,2104.0,975.0,2124.0,...,6361.942383,3565.255371,1878.698730,17489.666016,17283.833984,17201.283203,17021.105469,16690.128906,10616.111328,5948.324219
2,0,525.0,477.0,731.0,624.0,1090.0,3633.0,1726.0,895.0,2051.0,...,6153.169922,3460.817871,1826.467285,962.496521,765.000916,657.255798,257.338440,152.738708,69.780663,35.270126
3,0,577.0,446.0,699.0,625.0,1108.0,5036.0,1751.0,808.0,2287.0,...,10430.298828,5854.774414,3087.293457,7725.885254,7557.866211,7455.998047,7061.154297,6838.061035,4317.638672,2414.769287
4,0,541.0,432.0,759.0,579.0,1024.0,3126.0,2008.0,906.0,2159.0,...,23542.087891,13179.492188,6941.858398,11195.717773,11015.741211,10906.076172,10106.732422,9888.559570,6314.685059,3541.675293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78514,0,1208.0,405.0,1124.0,536.0,736.0,522.0,626.0,546.0,15176.0,...,2121.493652,1188.896484,626.485840,415.748596,266.493896,197.534225,27.509951,2.184534,0.766539,0.433579
78515,0,1240.0,394.0,1108.0,529.0,753.0,540.0,626.0,567.0,15334.0,...,7.271651,4.113052,2.175889,3639.195801,3455.077148,3390.203369,3334.017578,3263.763428,2090.744141,1173.604248
78516,0,1455.0,407.0,1077.0,521.0,754.0,549.0,588.0,580.0,19859.0,...,7.267348,4.110372,2.174457,28013.242188,27784.912109,27694.750000,27228.414062,26790.941406,17023.996094,9536.416016
78517,0,1483.0,401.0,948.0,546.0,819.0,547.0,563.0,597.0,20093.0,...,10665.289062,5972.991699,3146.582275,874.318542,654.879395,566.956238,34.042343,7.987534,5.287806,2.990932


In [8]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [127]:
path = './processed data/data_set_after_window_S64_withoutSC_allPa.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]
feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

threshold_WAMP:30.0, threshold_ZC:0.0, threshold_SSC:1.0,bins:9,ranges:(-70,70)
IEMG,MAV,SSI,VAR,RMS,WL,ZC,SSC,WAMP,skew,Acti,AR,HIST


1

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [222]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [223]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [4]:
from tensorflow.keras import layers,Model,callbacks
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

In [38]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        y_01[ind1] = 1
        cw = None
    else:
        ind = ((y==1)|(y==2)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:2,1:1.,2:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=False)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.2,
                                                       random_state=555,
                                                       shuffle=True)
    
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                        epochs=200,batch_size=50,class_weight=cw,
                        callbacks=[early_stopping])
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        y_pred_v=model.predict(x_valid)
        valid = metrics.accuracy_score(y_valid,y_pred_v>0.5)
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(y_train,y_pred_ta>0.5)
    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        y_pred_v=model.predict(x_valid)
        valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1)))
        print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,valid,test

def test_model(model,feature,y):
    ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
    y_01 = y.copy()
    y_01[ind] = 1
    y_pred=model.predict(feature)
    test = metrics.accuracy_score(y_01,y_pred>0.5)
    print('acc:%f'%test)
    return test

## ANN

In [224]:
#ind = ((y==1)|(y==2)|(y==3)|(y==6))
ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y.copy()
y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature_sc,y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [55]:
input_ = layers.Input(shape=feature_sc.shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
l3 = layers.Dense(32,activation='elu')(drop2)
drop3 = layers.Dropout(0.2)(l3)
l4 = layers.Dense(16,activation='elu')(drop3)
drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop4)
model = Model(inputs=[input_],outputs=[output])

In [56]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=200,batch_size=50,
                   callbacks=[early_stopping])

# Train model

In [57]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,valid,test = train_model(model,feature_sc,np.array(y),True)
rest = test_model(model,feature2_sc,np.array(y2))
acc = [train,valid,test,rest]

Train on 22313 samples, validate on 5579 samples
Epoch 1/200
22313/22313 [==============================] - 3s 128us/sample - loss: 0.3188 - accuracy: 0.8719 - val_loss: 0.2063 - val_accuracy: 0.9208
Epoch 2/200
22313/22313 [==============================] - 2s 73us/sample - loss: 0.2338 - accuracy: 0.9132 - val_loss: 0.1841 - val_accuracy: 0.9317
Epoch 3/200
22313/22313 [==============================] - 2s 70us/sample - loss: 0.2108 - accuracy: 0.9217 - val_loss: 0.1699 - val_accuracy: 0.9349
Epoch 4/200
22313/22313 [==============================] - 2s 72us/sample - loss: 0.1942 - accuracy: 0.9269 - val_loss: 0.1591 - val_accuracy: 0.9417
Epoch 5/200
22313/22313 [==============================] - 2s 74us/sample - loss: 0.1820 - accuracy: 0.9327 - val_loss: 0.1469 - val_accuracy: 0.9452
Epoch 6/200
22313/22313 [==============================] - 2s 72us/sample - loss: 0.1708 - accuracy: 0.9377 - val_loss: 0.1377 - val_accuracy: 0.9478
Epoch 7/200
22313/22313 [=========================

Epoch 55/200
22313/22313 [==============================] - 2s 70us/sample - loss: 0.0427 - accuracy: 0.9842 - val_loss: 0.0656 - val_accuracy: 0.9787
Epoch 56/200
22313/22313 [==============================] - 2s 70us/sample - loss: 0.0442 - accuracy: 0.9845 - val_loss: 0.0569 - val_accuracy: 0.9796
Epoch 57/200
22313/22313 [==============================] - 2s 69us/sample - loss: 0.0412 - accuracy: 0.9860 - val_loss: 0.0690 - val_accuracy: 0.9771
Epoch 58/200
22313/22313 [==============================] - 2s 69us/sample - loss: 0.0420 - accuracy: 0.9850 - val_loss: 0.0699 - val_accuracy: 0.9780
Epoch 59/200
22313/22313 [==============================] - 2s 78us/sample - loss: 0.0399 - accuracy: 0.9862 - val_loss: 0.0658 - val_accuracy: 0.9781
Epoch 60/200
22313/22313 [==============================] - 2s 69us/sample - loss: 0.0362 - accuracy: 0.9870 - val_loss: 0.0754 - val_accuracy: 0.9763
Epoch 61/200
22313/22313 [==============================] - 2s 70us/sample - loss: 0.0423 - ac

22313/22313 [==============================] - 2s 68us/sample - loss: 0.0229 - accuracy: 0.9921 - val_loss: 0.0602 - val_accuracy: 0.9835
Epoch 110/200
22313/22313 [==============================] - 1s 67us/sample - loss: 0.0254 - accuracy: 0.9911 - val_loss: 0.0586 - val_accuracy: 0.9833
Epoch 111/200
22313/22313 [==============================] - 2s 82us/sample - loss: 0.0242 - accuracy: 0.9917 - val_loss: 0.0524 - val_accuracy: 0.9846
Epoch 112/200
22313/22313 [==============================] - 2s 108us/sample - loss: 0.0204 - accuracy: 0.9931 - val_loss: 0.0558 - val_accuracy: 0.9833
Epoch 113/200
22313/22313 [==============================] - 2s 86us/sample - loss: 0.0222 - accuracy: 0.9926 - val_loss: 0.0550 - val_accuracy: 0.9808
test:0.943648
valid:0.985302
train:0.999417
acc:0.960683


In [54]:
test_model(model,feature2_sc,np.array(y2))

acc:0.961351


0.9613514827418571

In [39]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

array([[3498,  205,  215],
       [ 147, 6362,   61],
       [ 165,   58, 3178]], dtype=int64)

In [71]:
#acc_ann = pd.DataFrame([0.9995966476941693, 0.9758021150743861, 0.9736162890737023,0.9840787554691298],columns=['all_TT'],index=['train','valid','test','rest data'])
#acc_ann=acc_ann.T
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann

,train,valid,test,rest data
all_TT,0.999597,0.975802,0.973616,0.984079
drop_mDWT,0.999910,0.978849,0.978348,0.984524


# test feature from Left or Right

In [86]:
acc={}
cols = ['LEFT','RIGHT']

sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test = train_model(model,feature_sc,np.array(y))
    
    acc_rest=test_model(model,feature2_sc,np.array(y2))
    acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 22313 samples, validate on 5579 samples
Epoch 1/200
22313/22313 [==============================] - 2s 99us/sample - loss: 0.3881 - accuracy: 0.8370 - val_loss: 0.2879 - val_accuracy: 0.8901
Epoch 2/200
22313/22313 [==============================] - 1s 63us/sample - loss: 0.3186 - accuracy: 0.8696 - val_loss: 0.2700 - val_accuracy: 0.8953
Epoch 3/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.2965 - accuracy: 0.8804 - val_loss: 0.2495 - val_accuracy: 0.9046
Epoch 4/200
22313/22313 [==============================] - 1s 57us/sample - loss: 0.2836 - accuracy: 0.8871 - val_loss: 0.2379 - val_accuracy: 0.9072
Epoch 5/200
22313/22313 [==============================] - 1s 66us/sample - loss: 0.2686 - accuracy: 0.8953 - val_loss: 0.2283 - val_accuracy: 0.9120
Epoch 6/200
22313/22313 [==============================] - 1s 56us/sample - loss: 0.2626 - accuracy: 0.8985 - val_loss: 0.2242 - val_accuracy: 0.9161
Epoch 7/200
22313/22313 [==========================

Epoch 55/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.1405 - accuracy: 0.9469 - val_loss: 0.1475 - val_accuracy: 0.9469
Epoch 56/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.1398 - accuracy: 0.9465 - val_loss: 0.1416 - val_accuracy: 0.9471
Epoch 57/200
22313/22313 [==============================] - 1s 58us/sample - loss: 0.1387 - accuracy: 0.9491 - val_loss: 0.1482 - val_accuracy: 0.9441
Epoch 58/200
22313/22313 [==============================] - 1s 53us/sample - loss: 0.1408 - accuracy: 0.9469 - val_loss: 0.1416 - val_accuracy: 0.9443
Epoch 59/200
22313/22313 [==============================] - 1s 54us/sample - loss: 0.1371 - accuracy: 0.9486 - val_loss: 0.1397 - val_accuracy: 0.9441
Epoch 60/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.1342 - accuracy: 0.9502 - val_loss: 0.1420 - val_accuracy: 0.9441
Epoch 61/200
22313/22313 [==============================] - 1s 55us/sample - loss: 0.1354 - ac

22313/22313 [==============================] - 2s 80us/sample - loss: 0.1158 - accuracy: 0.9552 - val_loss: 0.1330 - val_accuracy: 0.9516
Epoch 77/200
22313/22313 [==============================] - 2s 80us/sample - loss: 0.1175 - accuracy: 0.9552 - val_loss: 0.1312 - val_accuracy: 0.9527
Epoch 78/200
22313/22313 [==============================] - 2s 78us/sample - loss: 0.1125 - accuracy: 0.9575 - val_loss: 0.1292 - val_accuracy: 0.9538
Epoch 79/200
22313/22313 [==============================] - 2s 79us/sample - loss: 0.1141 - accuracy: 0.9558 - val_loss: 0.1279 - val_accuracy: 0.9538
Epoch 80/200
22313/22313 [==============================] - 2s 75us/sample - loss: 0.1141 - accuracy: 0.9564 - val_loss: 0.1304 - val_accuracy: 0.9538
Epoch 81/200
22313/22313 [==============================] - 2s 80us/sample - loss: 0.1132 - accuracy: 0.9580 - val_loss: 0.1310 - val_accuracy: 0.9516
Epoch 82/200
22313/22313 [==============================] - 2s 75us/sample - loss: 0.1125 - accuracy: 0.957

In [47]:
pd.DataFrame(acc,index=['train','valid','test','rest data']).to_csv('./results/acc_lr_ann.csv')

,LEFT,RIGHT
train,0.987864,0.986136
valid,0.954364,0.951091
test,0.920873,0.904291
rest data,0.927044,0.894982


# test features from 2 of 8 signals

In [67]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test = train_model(model,feature_sc,np.array(y))
    acc_rest=test_model(model,feature2_sc,np.array(y2))
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test = train_model(model,feature_sc,np.array(y))
    acc_rest=test_model(model,feature2_sc,np.array(y2))
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 22000 samples, validate on 5500 samples
Epoch 1/200
22000/22000 [==============================] - 2s 93us/sample - loss: 0.2918 - accuracy: 0.8795 - val_loss: 0.2951 - val_accuracy: 0.8795
Epoch 2/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2900 - accuracy: 0.8787 - val_loss: 0.2929 - val_accuracy: 0.8813
Epoch 3/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2927 - accuracy: 0.8792 - val_loss: 0.2951 - val_accuracy: 0.8778
Epoch 4/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2918 - accuracy: 0.8788 - val_loss: 0.2927 - val_accuracy: 0.8804
Epoch 5/200
22000/22000 [==============================] - 1s 61us/sample - loss: 0.2853 - accuracy: 0.8810 - val_loss: 0.2954 - val_accuracy: 0.8773
Epoch 6/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.2922 - accuracy: 0.8788 - val_loss: 0.2912 - val_accuracy: 0.8793
Epoch 7/200
22000/22000 [==========================

Epoch 32/200
22000/22000 [==============================] - 1s 51us/sample - loss: 0.2643 - accuracy: 0.8957 - val_loss: 0.2539 - val_accuracy: 0.8984
Epoch 33/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2653 - accuracy: 0.8923 - val_loss: 0.2553 - val_accuracy: 0.8984
Epoch 34/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2624 - accuracy: 0.8951 - val_loss: 0.2546 - val_accuracy: 0.8989
Epoch 35/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2610 - accuracy: 0.8941 - val_loss: 0.2507 - val_accuracy: 0.8989
Epoch 36/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2653 - accuracy: 0.8940 - val_loss: 0.2509 - val_accuracy: 0.8980
Epoch 37/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2595 - accuracy: 0.8942 - val_loss: 0.2515 - val_accuracy: 0.9020
Epoch 38/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2605 - ac

Epoch 2/200
22000/22000 [==============================] - 1s 50us/sample - loss: 0.3814 - accuracy: 0.8343 - val_loss: 0.3601 - val_accuracy: 0.8460
Epoch 3/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.3690 - accuracy: 0.8378 - val_loss: 0.3516 - val_accuracy: 0.8525
Epoch 4/200
22000/22000 [==============================] - 1s 52us/sample - loss: 0.3555 - accuracy: 0.8460 - val_loss: 0.3409 - val_accuracy: 0.8587
Epoch 5/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3517 - accuracy: 0.8492 - val_loss: 0.3383 - val_accuracy: 0.8555
Epoch 6/200
22000/22000 [==============================] - 1s 45us/sample - loss: 0.3486 - accuracy: 0.8501 - val_loss: 0.3328 - val_accuracy: 0.8573
Epoch 7/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3387 - accuracy: 0.8545 - val_loss: 0.3308 - val_accuracy: 0.8602
Epoch 8/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3397 - accuracy:

22000/22000 [==============================] - 1s 45us/sample - loss: 0.2885 - accuracy: 0.8839 - val_loss: 0.2788 - val_accuracy: 0.8869
Epoch 33/200
22000/22000 [==============================] - 1s 44us/sample - loss: 0.2867 - accuracy: 0.8831 - val_loss: 0.2801 - val_accuracy: 0.8856
Epoch 34/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2852 - accuracy: 0.8864 - val_loss: 0.2775 - val_accuracy: 0.8876
Epoch 35/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2873 - accuracy: 0.8850 - val_loss: 0.2775 - val_accuracy: 0.8853
Epoch 36/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2847 - accuracy: 0.8878 - val_loss: 0.2754 - val_accuracy: 0.8871
Epoch 37/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2847 - accuracy: 0.8849 - val_loss: 0.2763 - val_accuracy: 0.8858
Epoch 38/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.2848 - accuracy: 0.885

22000/22000 [==============================] - 1s 48us/sample - loss: 0.3383 - accuracy: 0.8573 - val_loss: 0.3274 - val_accuracy: 0.8620
Epoch 15/200
22000/22000 [==============================] - 1s 42us/sample - loss: 0.3376 - accuracy: 0.8555 - val_loss: 0.3259 - val_accuracy: 0.8616
Epoch 16/200
22000/22000 [==============================] - 1s 42us/sample - loss: 0.3353 - accuracy: 0.8575 - val_loss: 0.3257 - val_accuracy: 0.8627
Epoch 17/200
22000/22000 [==============================] - 1s 42us/sample - loss: 0.3268 - accuracy: 0.8595 - val_loss: 0.3222 - val_accuracy: 0.8624
Epoch 18/200
22000/22000 [==============================] - 1s 42us/sample - loss: 0.3288 - accuracy: 0.8601 - val_loss: 0.3203 - val_accuracy: 0.8631
Epoch 19/200
22000/22000 [==============================] - 1s 42us/sample - loss: 0.3242 - accuracy: 0.8609 - val_loss: 0.3194 - val_accuracy: 0.8644
Epoch 20/200
22000/22000 [==============================] - 1s 43us/sample - loss: 0.3259 - accuracy: 0.861

...

22000/22000 [==============================] - 1s 47us/sample - loss: 0.3508 - accuracy: 0.8554 - val_loss: 0.3321 - val_accuracy: 0.8642
Epoch 6/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3452 - accuracy: 0.8542 - val_loss: 0.3255 - val_accuracy: 0.8671
Epoch 7/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3380 - accuracy: 0.8605 - val_loss: 0.3214 - val_accuracy: 0.8689
Epoch 8/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.3352 - accuracy: 0.8623 - val_loss: 0.3167 - val_accuracy: 0.8693
Epoch 9/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.3317 - accuracy: 0.8649 - val_loss: 0.3172 - val_accuracy: 0.8711
Epoch 10/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3246 - accuracy: 0.8685 - val_loss: 0.3123 - val_accuracy: 0.8731
Epoch 11/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.3219 - accuracy: 0.8704 - 

22000/22000 [==============================] - 1s 49us/sample - loss: 0.2233 - accuracy: 0.9130 - val_loss: 0.2627 - val_accuracy: 0.8995
Epoch 114/200
22000/22000 [==============================] - 1s 49us/sample - loss: 0.2259 - accuracy: 0.9127 - val_loss: 0.2610 - val_accuracy: 0.8993
Epoch 115/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2273 - accuracy: 0.9123 - val_loss: 0.2638 - val_accuracy: 0.8985
Epoch 116/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2277 - accuracy: 0.9129 - val_loss: 0.2598 - val_accuracy: 0.9002
Epoch 117/200
22000/22000 [==============================] - 1s 47us/sample - loss: 0.2249 - accuracy: 0.9136 - val_loss: 0.2617 - val_accuracy: 0.9005
Epoch 118/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2256 - accuracy: 0.9111 - val_loss: 0.2541 - val_accuracy: 0.9038
Epoch 119/200
22000/22000 [==============================] - 1s 46us/sample - loss: 0.2268 - accuracy:

22000/22000 [==============================] - 1s 58us/sample - loss: 0.2305 - accuracy: 0.9115 - val_loss: 0.2451 - val_accuracy: 0.9067
Epoch 79/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2322 - accuracy: 0.9095 - val_loss: 0.2470 - val_accuracy: 0.9053
Epoch 80/200
22000/22000 [==============================] - 1s 56us/sample - loss: 0.2307 - accuracy: 0.9131 - val_loss: 0.2492 - val_accuracy: 0.9056
Epoch 81/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2340 - accuracy: 0.9101 - val_loss: 0.2423 - val_accuracy: 0.9065
Epoch 82/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2304 - accuracy: 0.9120 - val_loss: 0.2449 - val_accuracy: 0.9073
Epoch 83/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2278 - accuracy: 0.9133 - val_loss: 0.2460 - val_accuracy: 0.9065
Epoch 84/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2307 - accuracy: 0.912

22000/22000 [==============================] - 1s 65us/sample - loss: 0.2245 - accuracy: 0.9135 - val_loss: 0.2362 - val_accuracy: 0.9100
Epoch 89/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2243 - accuracy: 0.9154 - val_loss: 0.2380 - val_accuracy: 0.9104
Epoch 90/200
22000/22000 [==============================] - 1s 60us/sample - loss: 0.2216 - accuracy: 0.9144 - val_loss: 0.2348 - val_accuracy: 0.9080
Epoch 91/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2267 - accuracy: 0.9135 - val_loss: 0.2366 - val_accuracy: 0.9085
Epoch 92/200
22000/22000 [==============================] - 1s 55us/sample - loss: 0.2219 - accuracy: 0.9137 - val_loss: 0.2333 - val_accuracy: 0.9111
Epoch 93/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2235 - accuracy: 0.9151 - val_loss: 0.2367 - val_accuracy: 0.9084
Epoch 94/200
22000/22000 [==============================] - 1s 54us/sample - loss: 0.2199 - accuracy: 0.916

22000/22000 [==============================] - 1s 53us/sample - loss: 0.2054 - accuracy: 0.9224 - val_loss: 0.2336 - val_accuracy: 0.9113
Epoch 143/200
22000/22000 [==============================] - 1s 57us/sample - loss: 0.2068 - accuracy: 0.9221 - val_loss: 0.2330 - val_accuracy: 0.9115
Epoch 144/200
22000/22000 [==============================] - 1s 53us/sample - loss: 0.2034 - accuracy: 0.9221 - val_loss: 0.2308 - val_accuracy: 0.9118
test:0.857455
valid:0.913273
train:0.948091
acc:0.848029
0.8480292480538952


In [73]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

,train,valid,test,rest data
LEFT_TA_LEFT_TS,0.912545,0.881273,0.789818,0.782118
LEFT_TA_LEFT_BF,0.931636,0.906909,0.904873,0.864276
LEFT_TA_LEFT_RF,0.915045,0.884545,0.859200,0.898761
LEFT_TS_LEFT_BF,0.933955,0.901636,0.848727,0.824697
LEFT_TS_LEFT_RF,0.933045,0.884182,0.848582,0.856040
LEFT_BF_LEFT_RF,0.927545,0.898909,0.876073,0.868178
RIGHT_TA_RIGHT_TS,0.928909,0.902000,0.865164,0.850987
RIGHT_TA_RIGHT_BF,0.938591,0.906000,0.872582,0.864995
RIGHT_TA_RIGHT_RF,0.928455,0.902545,0.877818,0.857539
RIGHT_TS_RIGHT_BF,0.945182,0.906182,0.900800,0.850289


# Test on some of rest

In [219]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

acc:0.997041
acc:0.994048
acc:0.956284
acc:0.980276
acc:0.991055
acc:0.906706
acc:0.856749
acc:0.966764
[0.9970414201183432, 0.9940476190476191, 0.9562841530054644, 0.980276134122288, 0.9910554561717353, 0.9067055393586005, 0.8567493112947658, 0.9667644183773216]


In [263]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

0.9589472691255542

## RNN

In [149]:
feature_re = feature_sc.reshape((-1,feature.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(feature_re,y_01,test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [156]:
input_ = layers.Input(shape=[feature.shape[1],1])
lstm1 = layers.GRU(100)(input_)
#lstm2 = layers.LSTM(20)(lstm1)
output = layers.Dense(1,activation='sigmoid')(lstm1)
model = Model(inputs=[input_],outputs=[output])

In [157]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [158]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

Train on 21990 samples, validate on 5498 samples
Epoch 1/100
21990/21990 [==============================] - 55s 3ms/sample - loss: 0.6715 - accuracy: 0.5768 - val_loss: 0.6168 - val_accuracy: 0.6808
Epoch 2/100
21990/21990 [==============================] - 52s 2ms/sample - loss: 0.5631 - accuracy: 0.7150 - val_loss: 0.5077 - val_accuracy: 0.7552
Epoch 3/100
21990/21990 [==============================] - 52s 2ms/sample - loss: 0.5141 - accuracy: 0.7546 - val_loss: 0.4896 - val_accuracy: 0.7696
Epoch 4/100
21990/21990 [==============================] - 51s 2ms/sample - loss: 0.4999 - accuracy: 0.7643 - val_loss: 0.4811 - val_accuracy: 0.7803
Epoch 5/100
21990/21990 [==============================] - 51s 2ms/sample - loss: 0.4728 - accuracy: 0.7853 - val_loss: 0.4479 - val_accuracy: 0.7968
Epoch 6/100
21990/21990 [==============================] - 54s 2ms/sample - loss: 0.4521 - accuracy: 0.7983 - val_loss: 0.4376 - val_accuracy: 0.8032
Epoch 7/100
21990/21990 [==========================

Epoch 55/100
21990/21990 [==============================] - 59s 3ms/sample - loss: 0.2446 - accuracy: 0.9028 - val_loss: 0.2694 - val_accuracy: 0.8909
Epoch 56/100
21990/21990 [==============================] - 60s 3ms/sample - loss: 0.2435 - accuracy: 0.9028 - val_loss: 0.2641 - val_accuracy: 0.8927
Epoch 57/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2359 - accuracy: 0.9075 - val_loss: 0.2672 - val_accuracy: 0.8927
Epoch 58/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2331 - accuracy: 0.9080 - val_loss: 0.2556 - val_accuracy: 0.8994
Epoch 59/100
21990/21990 [==============================] - 66s 3ms/sample - loss: 0.2314 - accuracy: 0.9074 - val_loss: 0.2608 - val_accuracy: 0.8920
Epoch 60/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2337 - accuracy: 0.9083 - val_loss: 0.2617 - val_accuracy: 0.8952
Epoch 61/100
21990/21990 [==============================] - 58s 3ms/sample - loss: 0.2263 - ac

In [159]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [160]:
test_model(model,feature2_re,y2)

acc:0.889541


0.8895405669599218